In [ ]:
from spyral_utils.plot import CutHandler, Histogrammer
from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.particle_id import serialize_particle_id, ParticleID
from spyral.core.constants import DEG2RAD
from spyral.core.run_stacks import form_run_string

import polars as pl
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from pathlib import Path

%matplotlib widget

RAD2DEG = 1.0/DEG2RAD

In [ ]:
# Load config
workspace_path = Path("D:\\e20009_analysis")
estimation_result_path = workspace_path / "Estimation"

# Set the run range (inclusive)
run_min = 347
run_max = 347

# IC gate
ic_min_val = -1.0
ic_max_val = 4095.0

# Nucleus map
nuclear_map = NuclearDataMap()

In [ ]:
# Set PID cut parameters for nucleus that will be gated on
pid_name = "proton_cut" # name given to PID
pid_z = 1 # atomic number
pid_a = 1 # mass number
nucleus = nuclear_map.get_data(pid_z, pid_a)
pid_x_axis = "sqrt_dEdx" # This is the PID x-axis, matching a column name in the estimation dataframe
pid_y_axis = "brho" # This is the PID y-axis, matching a column name in the estimation dataframe
pid_path = Path("C:\\Users\\zachs\\Desktop\\particle_id.json") # Path to which we will write our PID


In [ ]:
# Make utility objects for plotting and making cuts
grammer = Histogrammer()
handler = CutHandler()

In [ ]:
# Create histograms
grammer.add_hist2d("particle_id", (400, 400), ((-10.0, 100), (-0.1, 2.5))) # Plot of dEdx vs. Brho (particle ID)
grammer.add_hist1d("ion_chamber", 4095, (-1.0, 4095.0)) # Plot of ion chamber (beam ID)
grammer.add_hist2d("kinematics", (720, 400), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics)

In [ ]:
# Fill histograms
for run in range(run_min, run_max+1):
    run_path = estimation_result_path / f"{form_run_string(run)}.parquet"
    if not run_path.exists():
        continue
    df = pl.scan_parquet(run_path)

    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > ic_min_val) & (pl.col('ic_amplitude') < ic_max_val))
    df = df.collect()

    grammer.fill_hist2d('particle_id', df.select(pid_x_axis).to_numpy(), df.select(pid_y_axis).to_numpy()) # Use your custom axes
    grammer.fill_hist2d('kinematics', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())
    grammer.fill_hist1d('ion_chamber', df.unique(subset=['event']).select('ic_amplitude').to_numpy())

In [ ]:
# Plot IC
ic = grammer.get_hist1d("ion_chamber")
fig, ax = plt.subplots(1,1)
ax.stairs(ic.counts, edges=ic.bins)
ax.set_title("Ion Chamber")
ax.set_xlabel("Amplitude (arb.)")
ax.set_ylabel("Counts")
fig.set_figwidth(8.0)

In [ ]:
# Plot PID and draw gate by clicking points on plot
pid_hist = grammer.get_hist2d("particle_id")
fig, ax = plt.subplots(1,1)
_ = PolygonSelector(ax, handler.mpl_on_select)
mesh = ax.pcolormesh(pid_hist.x_bins, pid_hist.y_bins, pid_hist.counts, norm='log')
fig.colorbar(mesh, ax=ax)
ax.set_title("Particle ID")
ax.set_xlabel(f"{pid_x_axis} Column")
ax.set_ylabel(f"{pid_y_axis} Column")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)


In [ ]:
# Plot estimated kinematics
kine = grammer.get_hist2d("kinematics")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(kine.x_bins, kine.y_bins, kine.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Kinematics")
ax.set_xlabel("Polar angle $\\theta$ (deg)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)

In [ ]:
# Save PID
# If you've made multiple cuts, you'll want to change the name used here. Cuts are automatically named in the order they were made (first cut is cut_0, second cut_1, etc.)
cut = handler.cuts["cut_0"]
cut.name = pid_name
cut.x_axis = pid_x_axis # specify our axes
cut.y_axis = pid_y_axis
pid = ParticleID(cut, nucleus)
# Save our pid
serialize_particle_id(pid_path, pid)


In [ ]:
# Make and fill kinematics and PID histograms gated on user input gate
grammer.add_hist2d("particle_id_gated", (400, 400), ((-10.0, 100.0), (-0.1, 2.5))) # Plot of dEdx vs. Brho (particle ID), gated on PID
grammer.add_hist2d("kinematics_gated", (720, 400), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics), gated on PID

for run in range(run_min, run_max+1):
    run_path = estimation_result_path / f"{form_run_string(run)}.parquet"
    if not run_path.exists():
        continue
    df = pl.scan_parquet(run_path)

    df = df.filter(pl.struct([pid_x_axis, pid_y_axis]).map_batches(pid.cut.is_cols_inside)) # apply PID
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > ic_min_val) & (pl.col('ic_amplitude') < ic_max_val))
    df = df.collect()

    grammer.fill_hist2d('particle_id_gated', df.select(pid_x_axis).to_numpy(), df.select(pid_y_axis).to_numpy())
    grammer.fill_hist2d('kinematics_gated', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())

In [ ]:
# Plot gated PID
pid_gated = grammer.get_hist2d("particle_id_gated")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(pid_gated.x_bins, pid_gated.y_bins, pid_gated.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Particle ID Gated")
ax.set_xlabel(f"{pid_x_axis} Column")
ax.set_ylabel(f"{pid_y_axis} Column")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)

In [ ]:
# Plot gated kinematics
kine_gated = grammer.get_hist2d("kinematics_gated")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(kine_gated.x_bins, kine_gated.y_bins, kine_gated.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Kinematics Gated")
ax.set_xlabel("Polar angle $\\theta$ (deg)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)